In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import explained_variance_score,mean_squared_error,mean_absolute_error
import matplotlib.pyplot as plt
from keras.layers import *
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from keras.models import load_model
import seaborn as sns

In [ ]:
dataset = pd.read_csv("weather_2017+2018_hourly.csv")
data_as_np = dataset.GHI.values

In [ ]:
def series_to_supervised(data_np,n_lags,n_outs):
    samples = len(data_np)-(n_lags+n_outs-1)
    Inputs = []
    Targets = np.empty((samples,n_outs))
    for i in range(samples):
        step = i+n_lags
        sample = data_np[i:step]
        Inputs.append(sample)
        label = data_np[step:(step+n_outs)]
        j = 0
        for a in label:
            Targets[i,j] = a
            j = j+1
    return np.array(Inputs),Targets

In [ ]:
def plot_model_history(model_summary):
    plt.plot(model_summary.history['loss'])
    plt.plot(model_summary.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train','test'],loc='upper right')
    plt.show()

In [ ]:
def rmse_error(pred,y):
    return np.sqrt(mean_squared_error(pred,y))

In [ ]:
def create_model(input_shape,n_out,train_x,train_y,test_x,test_y,rate,filter):

    model = Sequential()
    model.add(ConvLSTM2D(filters = filter, kernel_size=(1,2), activation='relu', input_shape= input_shape))
    model.add(Flatten())

  #  output layer
    model.add(Dense(n_out))

    cp1 = ModelCheckpoint('ConvLSTM/', save_best_only=True)
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate = rate), metrics=[RootMeanSquaredError()])
    model_summary = model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=50, batch_size=64, callbacks=[cp1])
    plot_model_history(model_summary)

    model = load_model('ConvLSTM/')

    return model

In [ ]:
train_rmse_error, test_rmse_error, test_mae = 0,0,0
data_np = data_as_np.reshape(-1,1)
lags = [120,240]
seqs = [5]
filters = [64]
learning_rate = [0.01,0.001,0.0001,0.00001]
n_out = 24
# Prepare data
for n_lags in lags:
    Inputs,Targets = series_to_supervised(data_np,n_lags,n_out)
    for n_seq in seqs:
        n_steps = int(n_lags/n_seq)
        Inputs = Inputs.reshape((Inputs.shape[0],n_seq,1,n_steps,1))
        Inputs_model, Targets_model = Inputs[:-10], Targets[:-10]
        Inputs_plot, Targets_plot = Inputs[-10:], Targets[-10:]

        no_samples = Targets_model.shape[0]
        split = int(0.3 * no_samples)

        input_shape = (n_seq,1,n_steps,1)

        train_x,train_y = Inputs_model[:-split],Targets_model[:-split]
        test_x,test_y = Inputs_model[-split:],Targets_model[-split:]
        for filterr in filters:
            for rate in learning_rate:
                for _ in range(2):
                    model = create_model(input_shape,n_out,train_x,train_y,test_x,test_y,rate,filterr)

                    train_pred,test_pred = model.predict(train_x), model.predict(test_x)

                    train_rmse_error += rmse_error(train_pred.flatten(),train_y.flatten())
                    test_rmse_error += rmse_error(test_pred.flatten(),test_y.flatten())
                    test_mae += mean_absolute_error(test_pred.flatten(),test_y.flatten())
                with open("ConvLSTM.txt","a") as f:
                    f.write(f"{n_lags},{n_seq},{rate},{filterr},{train_rmse_error/2},{test_rmse_error/2},{test_mae/2}\n")